In [2]:
import os
import json
import spacy
from collections import defaultdict

# Laden des deutschen Modells von spaCy
nlp = spacy.load("de_core_news_sm")

def get_adjective_ratio(text):
    doc = nlp(text)
    total_tokens = len(doc)
    adjectives = sum(1 for token in doc if token.pos_ == "ADJ")
    return adjectives / total_tokens if total_tokens > 0 else 0

def process_folder(folder_path):
    ratios = defaultdict(list)
    for filename in os.listdir(folder_path):
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
            data = json.load(file)
            poem_text = " ".join([line["text"] for line in data["poem"].values()])
            ratio = get_adjective_ratio(poem_text)
            decade = (int(data["author_birth"]) // 10) * 10
            ratios[decade].append(ratio)
    return ratios

def average_ratios(ratios):
    return {decade: (sum(values) / len(values), len(values)) for decade, values in ratios.items() if len(values) > 0}

male_ratios = process_folder(r"D:\!vinzent\Uni\DH\Forschun gsmethoden\Hausarbeit\korpus neu\m")
female_ratios = process_folder(r"D:\!vinzent\Uni\DH\Forschun gsmethoden\Hausarbeit\korpus neu\w")

male_avg_ratios = average_ratios(male_ratios)
female_avg_ratios = average_ratios(female_ratios)

# Durchschnitt für den gesamten Korpus (m/w getrennt und gesamt)
all_male_ratios = [ratio for ratios in male_ratios.values() for ratio in ratios]
all_female_ratios = [ratio for ratios in female_ratios.values() for ratio in ratios]
all_ratios = all_male_ratios + all_female_ratios

print(f"Durchschnittliche Frequenz von Adjektiven für den gesamten Korpus (Männer): {sum(all_male_ratios) / len(all_male_ratios):.2f}")
print(f"Durchschnittliche Frequenz von Adjektiven für den gesamten Korpus (Frauen): {sum(all_female_ratios) / len(all_female_ratios):.2f}")
print(f"Durchschnittliche Frequenz von Adjektiven für den gesamten Korpus (gesamt): {sum(all_ratios) / len(all_ratios):.2f}")
print()

print("Durchschnittliche Frequenz von Adjektiven für Männer:")
for decade in sorted(male_avg_ratios.keys()):
    avg, count = male_avg_ratios[decade]
    print(f"{decade}s: {avg:.2f} (Anzahl der Gedichte: {count})")

print("\nDurchschnittliche Frequenz von Adjektiven für Frauen:")
for decade in sorted(female_avg_ratios.keys()):
    avg, count = female_avg_ratios[decade]
    print(f"{decade}s: {avg:.2f} (Anzahl der Gedichte: {count})")


Durchschnittliche Frequenz von Adjektiven für den gesamten Korpus (Männer): 0.04
Durchschnittliche Frequenz von Adjektiven für den gesamten Korpus (Frauen): 0.04
Durchschnittliche Frequenz von Adjektiven für den gesamten Korpus (gesamt): 0.04

Durchschnittliche Frequenz von Adjektiven für Männer:
0s: 0.03 (Anzahl der Gedichte: 22)
1870s: 0.05 (Anzahl der Gedichte: 7)
1880s: 0.05 (Anzahl der Gedichte: 9)
1890s: 0.03 (Anzahl der Gedichte: 22)
1900s: 0.04 (Anzahl der Gedichte: 13)
1920s: 0.03 (Anzahl der Gedichte: 98)
1930s: 0.03 (Anzahl der Gedichte: 189)
1940s: 0.03 (Anzahl der Gedichte: 212)
1950s: 0.04 (Anzahl der Gedichte: 220)
1960s: 0.04 (Anzahl der Gedichte: 328)
1970s: 0.04 (Anzahl der Gedichte: 247)
1980s: 0.04 (Anzahl der Gedichte: 107)
1990s: 0.03 (Anzahl der Gedichte: 8)
2010s: 0.00 (Anzahl der Gedichte: 1)

Durchschnittliche Frequenz von Adjektiven für Frauen:
0s: 0.06 (Anzahl der Gedichte: 33)
1900s: 0.04 (Anzahl der Gedichte: 22)
1910s: 0.03 (Anzahl der Gedichte: 9)
1920s:

In [4]:
from scipy.stats import ttest_ind, mannwhitneyu, shapiro

def perform_tests(data_m, data_w, sample_name):
    if len(data_m) < 3 or len(data_w) < 3:
        print(f"Tests für {sample_name} können nicht durchgeführt werden, da weniger als 3 Datenpunkte für mindestens eine der Gruppen vorhanden sind.")
        return

    # Überprüfen, ob die Daten normalverteilt sind
    _, p_value_m = shapiro(data_m)
    _, p_value_w = shapiro(data_w)
    alpha = 0.05

    # Wenn beide Daten normalverteilt sind, führen Sie den t-Test durch
    if p_value_m > alpha and p_value_w > alpha:
        stat, p_value = ttest_ind(data_m, data_w)
        print(f"t-Test für {sample_name}:")
    # Andernfalls führen Sie den Whitney U-Test durch
    else:
        stat, p_value = mannwhitneyu(data_m, data_w)
        print(f"Whitney U-Test für {sample_name}:")

    print("Teststatistik:", stat)
    print("p-Wert:", p_value)
    if p_value < alpha:
        print(f"Es gibt einen signifikanten Unterschied in den Adjektivfrequenzen zwischen Männern und Frauen für {sample_name}.")
    else:
        print(f"Es gibt keinen signifikanten Unterschied in den Adjektivfrequenzen zwischen Männern und Frauen für {sample_name}.")
    print()

# Durchführung der Tests für den gesamten Korpus
all_male_ratios = [ratio for ratios in male_ratios.values() for ratio in ratios]
all_female_ratios = [ratio for ratios in female_ratios.values() for ratio in ratios]
perform_tests(all_male_ratios, all_female_ratios, "gesamter Korpus")

# Durchführung der Tests für jedes Jahrzehnt
for decade in sorted(male_avg_ratios.keys()):
    male_decade_frequencies = [ratio for ratio in male_ratios[decade]]
    female_decade_frequencies = [ratio for ratio in female_ratios.get(decade, [])]  # Verwenden Sie get(), um einen Standardwert zurückzugeben, wenn das Jahrzehnt nicht vorhanden ist
    perform_tests(male_decade_frequencies, female_decade_frequencies, f"{decade}s")


Whitney U-Test für gesamter Korpus:
Teststatistik: 664928.0
p-Wert: 1.0215020087924686e-05
Es gibt einen signifikanten Unterschied in den Adjektivfrequenzen zwischen Männern und Frauen für gesamter Korpus.

t-Test für 0s:
Teststatistik: -2.966729080159608
p-Wert: 0.004507350658566995
Es gibt einen signifikanten Unterschied in den Adjektivfrequenzen zwischen Männern und Frauen für 0s.

Tests für 1870s können nicht durchgeführt werden, da weniger als 3 Datenpunkte für mindestens eine der Gruppen vorhanden sind.
Tests für 1880s können nicht durchgeführt werden, da weniger als 3 Datenpunkte für mindestens eine der Gruppen vorhanden sind.
Tests für 1890s können nicht durchgeführt werden, da weniger als 3 Datenpunkte für mindestens eine der Gruppen vorhanden sind.
t-Test für 1900s:
Teststatistik: 0.2039149133313595
p-Wert: 0.8396723261999965
Es gibt keinen signifikanten Unterschied in den Adjektivfrequenzen zwischen Männern und Frauen für 1900s.

Whitney U-Test für 1920s:
Teststatistik: 1527